In [1]:
import cv2
import os
import numpy as np
import shutil

In [2]:
label_root = 'C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Label/'
root = 'C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Merge/'
save_root = 'C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/230107_3/'
#save_root2 = 'C:/Users/glad/Desktop/졸업/Trim_data_2s/'
#file_list = []
#image_list = []
fourcc = cv2.VideoWriter_fourcc(*'mp4v')


In [3]:
file_list = os.listdir(root)
label_list = os.listdir(label_root)
gt_list = []

In [4]:
print(len(file_list))
print(len(label_list))

832
832


In [5]:
file_list[0][8]

'A'

In [6]:
import time
ratio = 2 # get ratio +1
x_offset = 0
x_offset_2 = 0
y_offset = 0
y_offset_2 = 0
for i in range(len(file_list)):
    image_list = []
    
    #cap.release()
    #out.release()
    video_file = root+file_list[i] # 동영상 파일 경로
    txt_file = label_root+label_list[i]
    print(video_file)
    print(txt_file)
    cap = cv2.VideoCapture(video_file) # 동영상 캡쳐 객체 생성  ---①
    video_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("length:",video_w,video_h)
    if file_list[i][8] == "A":
        with open(txt_file,'r') as f:
            line = f.readline()
            while line:

                if line.split(',')[0] == "A":
                    obj_id = line.split(',')[1]
                    break
                else:
                    line = f.readline()
                
        with open(txt_file,'r') as f:
            line = f.readline()
            while line:
                obj_id2,x1,y1,x2,y2 = line.split(',')[1:]
                if obj_id2 == obj_id:
                    break
                else:
                    line = f.readline()
    else:
        with open(txt_file,'r') as f:
            line = f.readline()
            if not line:
                break
            obj_id,x1,y1,x2,y2 = line.split(',')[1:]
            line = f.readline()
            obj_id2,x12,y12,x22,y22 = line.split(',')[1:]


    print(obj_id,x1,y1,x2,y2)
    obj_id,x1,y1,x2,y2 = [int(x) for x in [obj_id,x1,y1,x2,y2]]
    w = int(x2+(x2-x1)*ratio/2) - int(x1-(x2-x1)*ratio/2)
    h = int(y2+(y2-y1)*ratio/2) - int(y1-(y2-y1)*ratio/2)
    
    print(int(x1-(x2-x1)*ratio/2),int(x2+(x2-x1)*ratio/2),int(y1-(y2-y1)*ratio/2),int(y2+(y2-y1)*ratio/2))
    print("w,h",w,h)
    print(int(x1-(x2-x1)*ratio/2))
    print(int(y1-(y2-y1)*ratio/2))
    if int(x1-(x2-x1)*ratio/2) < 0:
        x_offset = -int(x1-(x2-x1)*ratio/2)
    else:
        x_offset = 0
    if int(x2+(x2-x1)*ratio/2) > video_w:
        x_offset_2 = int(x2+(x2-x1)*ratio/2)-video_w
    else:
        x_offset_2 = 0
    if int(y2+(y2-y1)*ratio/2) > video_h:
        y_offset_2 = int(y2+(y2-y1)*ratio/2)-video_h
    else:
        y_offset_2 = 0
    if int(y1-(y2-y1)*ratio/2) < 0:
        y_offset = -int(y1-(y2-y1)*ratio/2)
    else:
        y_offset = 0
    #x_offset =
    #y_offset =
    base = np.zeros([h,w,3])
    print("x_offset,y_offset",x_offset,y_offset)
    print("x_offset_2,y_offest_2",x_offset_2,y_offset_2)
    if cap.isOpened(): # 캡쳐 객체 초기화 확인
        #w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        #h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        while True:

            
            ret, img = cap.read()      # 다음 프레임 읽기      --- ②
            #print(img.shape)
            #print(img)e9
            
            if ret:                     # 프레임 읽기 정상
                #print(int(y1-(y2-y1)*ratio/2)+y_offset:int(y2+(y2-y1)*ratio/2)-y_offset_2,int(x1-(x2-x1)*ratio/2)+x_offset:int(x2+(x2-x1)*ratio/2)-x_offset_2)
                img = img[(int(y1-(y2-y1)*ratio/2)+y_offset):(int(y2+(y2-y1)*ratio/2)-y_offset_2),(int(x1-(x2-x1)*ratio/2)+x_offset):(int(x2+(x2-x1)*ratio/2)-x_offset_2)]
                y,x,c = img.shape[0],img.shape[1],img.shape[2]
                #print(y,x,c)
                #print(h-y_offset-y_offset_2)
                #img = np.concatenate([np.zeros([y_offset,w-x_offset-x_offset_2,3]),img],axis=0)
                img = np.concatenate([np.zeros([y_offset,x,3],dtype=np.uint8),img],axis=0)
                y,x,c = img.shape[0],img.shape[1],img.shape[2]
                #img = np.concatenate([img,np.zeros([y_offset_2,w-x_offset-x_offset_2,3])],axis=0)
                img = np.concatenate([img,np.zeros([y_offset_2,x,3],dtype=np.uint8)],axis=0)
                y,x,c = img.shape[0],img.shape[1],img.shape[2]
                #img = np.concatenate([np.zeros([h-y_offset-y_offset_2,x_offset,3]),img],axis=1)
                img = np.concatenate([np.zeros([y,x_offset,3],dtype=np.uint8),img],axis=1)
                y,x,c = img.shape[0],img.shape[1],img.shape[2]
                img = np.concatenate([img,np.zeros([y,x_offset_2,3],dtype=np.uint8)],axis=1)
                #img = np.concatenate([img,np.zeros([h-y_offset-y_offset_2,x_offset_2,3])],axis=1)
                
                #print(img.dtype)
                #print(img.shape)
                #img = cv2.add(img,base,dtype = cv2.CV_8U)
                #img = base + img +img
                #print(img)
                #print(base.shape)
                #print(img.shape)
                #w = int(x2)-int(x1)
                #h = int(y2)-int(y1)
                
                #w = int(x2+(x2-x1)*ratio/2) - int(x1-(x2-x1)*ratio/2)
                #h = int(y2+(y2-y1)*ratio/2) - int(y1-(y2-y1)*ratio/2)
                #img = img[145:490,0:545]
                image_list.append(img)
            else:                       # 다음 프레임 읽을 수 없슴,
                break                   # 재생 완료
    else:
        print("can't open video.")      # 캡쳐 객체 초기화 실패
    #image_list = image_list[-15:]
    print(len(image_list))
    cap.release()                       # 캡쳐 자원 반납
    out = cv2.VideoWriter(save_root+file_list[i].split('.')[0]+"_"+str(obj_id)+'.mp4', fourcc, 30, (w, h))
    for k in image_list:
        out.write(k)
    out.release()
    cap = cv2.VideoCapture(video_file) # 동영상 캡쳐 객체 생성  ---①
    if not file_list[i][8] == "A":
        obj_id,x1,y1,x2,y2=obj_id2,x12,y12,x22,y22
        print(obj_id,x1,y1,x2,y2)
        obj_id,x1,y1,x2,y2 = [int(x) for x in [obj_id,x1,y1,x2,y2]]
        w = int(x2+(x2-x1)*ratio/2) - int(x1-(x2-x1)*ratio/2)
        h = int(y2+(y2-y1)*ratio/2) - int(y1-(y2-y1)*ratio/2)
        
        print(int(x1-(x2-x1)*ratio/2),int(x2+(x2-x1)*ratio/2),int(y1-(y2-y1)*ratio/2),int(y2+(y2-y1)*ratio/2))
        print("w,h",w,h)
        print(int(x1-(x2-x1)*ratio/2))
        print(int(y1-(y2-y1)*ratio/2))
        if int(x1-(x2-x1)*ratio/2) < 0:
            x_offset = -int(x1-(x2-x1)*ratio/2)
        else:
            x_offset = 0
        if int(x2+(x2-x1)*ratio/2) > video_w:
            x_offset_2 = int(x2+(x2-x1)*ratio/2)-video_w
        else:
            x_offset_2 = 0
        if int(y2+(y2-y1)*ratio/2) > video_h:
            y_offset_2 = int(y2+(y2-y1)*ratio/2)-video_h
        else:
            y_offset_2 = 0
        if int(y1-(y2-y1)*ratio/2) < 0:
            y_offset = -int(y1-(y2-y1)*ratio/2)
        else:
            y_offset = 0
        #x_offset =
        #y_offset =
        base = np.zeros([h,w,3])
        print("x_offset,y_offset",x_offset,y_offset)
        print("x_offset_2,y_offest_2",x_offset_2,y_offset_2)
        if cap.isOpened(): # 캡쳐 객체 초기화 확인
            #w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            #h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            while True:

                
                ret, img = cap.read()      # 다음 프레임 읽기      --- ②
                #print(img.shape)
                #print(img)
                
                if ret:                     # 프레임 읽기 정상
                    #print(int(y1-(y2-y1)*ratio/2)+y_offset:int(y2+(y2-y1)*ratio/2)-y_offset_2,int(x1-(x2-x1)*ratio/2)+x_offset:int(x2+(x2-x1)*ratio/2)-x_offset_2)
                    img = img[(int(y1-(y2-y1)*ratio/2)+y_offset):(int(y2+(y2-y1)*ratio/2)-y_offset_2),(int(x1-(x2-x1)*ratio/2)+x_offset):(int(x2+(x2-x1)*ratio/2)-x_offset_2)]
                    y,x,c = img.shape[0],img.shape[1],img.shape[2]
                    #print(y,x,c)
                    #print(h-y_offset-y_offset_2)
                    #img = np.concatenate([np.zeros([y_offset,w-x_offset-x_offset_2,3]),img],axis=0)
                    img = np.concatenate([np.zeros([y_offset,x,3],dtype=np.uint8),img],axis=0)
                    y,x,c = img.shape[0],img.shape[1],img.shape[2]
                    #img = np.concatenate([img,np.zeros([y_offset_2,w-x_offset-x_offset_2,3])],axis=0)
                    img = np.concatenate([img,np.zeros([y_offset_2,x,3],dtype=np.uint8)],axis=0)
                    y,x,c = img.shape[0],img.shape[1],img.shape[2]
                    #img = np.concatenate([np.zeros([h-y_offset-y_offset_2,x_offset,3]),img],axis=1)
                    img = np.concatenate([np.zeros([y,x_offset,3],dtype=np.uint8),img],axis=1)
                    y,x,c = img.shape[0],img.shape[1],img.shape[2]
                    img = np.concatenate([img,np.zeros([y,x_offset_2,3],dtype=np.uint8)],axis=1)
                    #img = np.concatenate([img,np.zeros([h-y_offset-y_offset_2,x_offset_2,3])],axis=1)
                    
                    #print(img.dtype)
                    #print(img.shape)
                    #img = cv2.add(img,base,dtype = cv2.CV_8U)
                    #img = base + img +img
                    #print(img)
                    #print(base.shape)
                    #print(img.shape)
                    #w = int(x2)-int(x1)
                    #h = int(y2)-int(y1)
                    
                    #w = int(x2+(x2-x1)*ratio/2) - int(x1-(x2-x1)*ratio/2)
                    #h = int(y2+(y2-y1)*ratio/2) - int(y1-(y2-y1)*ratio/2)
                    #img = img[145:490,0:545]
                    image_list.append(img)
                else:                       # 다음 프레임 읽을 수 없슴,
                    break                   # 재생 완료
        else:
            print("can't open video.")      # 캡쳐 객체 초기화 실패
        #image_list = image_list[-15:]
        print(len(image_list))
        cap.release()                       # 캡쳐 자원 반납
        out = cv2.VideoWriter(save_root+file_list[i].split('.')[0]+"_"+str(obj_id)+'.mp4', fourcc, 30, (w, h))
        for k in image_list:
            out.write(k)
        out.release()
            
        #time.sleep(1)
    

C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Merge/220510_LA_0000.mp4
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Label/220510_LA_0000.txt
length: 1920 1080
1 92 363 532 662

-348 972 64 961
w,h 1320 897
-348
64
x_offset,y_offset 348 0
x_offset_2,y_offest_2 0 0
90
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Merge/220510_LA_0001.mp4
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Label/220510_LA_0001.txt
length: 1920 1080
0 436 237 767 419

105 1098 55 601
w,h 993 546
105
55
x_offset,y_offset 0 0
x_offset_2,y_offest_2 0 0
91
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Merge/220510_LA_0002.mp4
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Label/220510_LA_0002.txt
length: 1920 1080
0 427 236 754 418

100 1081 54 600
w,h 981 546
100
54
x_offset,y_offset 0 0
x_offset_2,y_offest_2 0 0
92
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Merge/220510_LA_0003.mp4
C:/Users/glad/Desktop/데이터공개/공개용 데이터셋/Label/220510_LA_0003.txt
length: 1920 1080
0 449 235 778 416

120 1107 54 597
w,h 987 543
120
54
x_offset,y_offset 0 0
x_offset_2,y_offest_2 0 0
91
C:/Users/

In [7]:
file_list[832]

IndexError: list index out of range